# PyCity Schools Analysis

- **Observable Trend 1**: The top 5 performing schools are all charter schools, the bottom 5 are all district schools
- **Observable Trend 2**: The average reading score is higher than the average math score at every school in the city
- **Observable Trend 3**: District schools performed far worse across the board in math and reading scores than charter schools <br>
  (And... the lowest scores were also observed among schools that spend the most per student, and the largest schools)

In [1]:
#import dependencies

import pandas as pd 
import numpy as np

In [2]:
#read in the school data and have a first look

school_file = "raw_data/schools_complete.csv"
schools_df = pd.read_csv(school_file)
schools_df.head()

,School ID,name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [3]:
#read in the student file and have a first look 

student_file = "raw_data/students_complete.csv"
students_df = pd.read_csv(student_file)
students_df.head()

,Student ID,name,gender,grade,school,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


# DISTRICT SUMMARY

In [4]:
#If I want to set index to school id, just do schools_raw.set_index("School ID")
Total_schools = len(schools_df["name"].unique())

Total_students = schools_df["size"].sum()

Total_budget = schools_df["budget"].sum()
# now we'll have to mask in currency form

avg_math = students_df["math_score"].sum()/students_df["name"].count()
avg_math = round(avg_math, 2)

avg_reading = students_df["reading_score"].sum()/students_df["name"].count()
avg_reading = round(avg_reading, 2)

passing_math = len(students_df.loc[students_df["math_score"]>70])/len(students_df["name"])*100
passing_math = round(passing_math, 2)

passing_reading = len(students_df.loc[students_df["reading_score"]>70])/len(students_df["name"])*100
passing_reading = round(passing_reading, 2)

overall_passing_rate = (passing_math +  passing_reading) / 2

# Create the table
Dist_summary = pd.DataFrame({"Total Schools":[Total_schools], "Total Students":[Total_students], 
                             "Total Budget":[Total_budget], "Average Math Score":[avg_math], 
                             "Average Reading Score":[avg_reading], "% Passing Math": [passing_math],
                            "% Passing Reading": [passing_reading], "% Overall Passing Rate":[overall_passing_rate]})

Dist_summary["Total Students"] = Dist_summary["Total Students"].map("{:,}".format)
Dist_summary["Total Budget"] = Dist_summary["Total Budget"].map("${:,.2f}".format)

org_dist_summary = Dist_summary[["Total Schools", "Total Students", "Total Budget", "Average Math Score", "Average Reading Score",
                                "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]]

org_dist_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.99,81.88,72.39,82.97,77.68


# SCHOOL SUMMARY

In [5]:
#Start with the schools data frame we have and clean it up
schools_df.head()
schools_df.set_index("School ID")
schools_summ = schools_df.rename(columns={"name": "School Name", "type": "School Type", "size": "Total Students", 
                                          "budget":"Total School Budget"})
#Create the Per Student Budget Column
schools_summ["Per Student Budget"] = schools_summ["Total School Budget"] / schools_summ["Total Students"]
schools_summ = schools_summ.set_index("School ID")

In [6]:
#Find the average and passing scores for each school, store them in lists
av_m = []
av_r = []
pass_m = []
pass_r = []
overall = []
for school in schools_summ["School Name"]:
    my_school_frame = students_df.loc[students_df["school"]==school,:]
    av_m.append(my_school_frame["math_score"].sum()/my_school_frame["name"].count())
    av_r.append(my_school_frame["reading_score"].sum()/my_school_frame["name"].count())
    pass_math = len(my_school_frame.loc[my_school_frame["math_score"]>70])/len(my_school_frame["name"])*100
    pass_m.append(pass_math)   
    pass_read = len(my_school_frame.loc[my_school_frame["reading_score"]>70])/len(my_school_frame["name"])*100
    pass_r.append(pass_read)
    overall.append((pass_math + pass_read)/2)

In [7]:
# Now append those lists to our data frame, and set the index as the school name to match the presentation image
schools_summ["Average Math Score"] = av_m
schools_summ["Average Reading Score"] = av_r
schools_summ["% Passing Math"] = pass_m
schools_summ["% Passing Reading"] = pass_r
schools_summ["% Overall Passing Rate"] = overall
schools_summ_present = schools_summ.set_index("School Name")
schools_summ_present["Total Students"] = schools_summ_present["Total Students"].map("{:,}".format)
schools_summ_present["Total School Budget"] = schools_summ_present["Total School Budget"].map("${:,.2f}".format)
schools_summ_present["Per Student Budget"] = schools_summ_present["Per Student Budget"].map("${:,.2f}".format)
del schools_summ_present.index.name
schools_summ_present

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,63.318478,78.813850,71.066164
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,63.750424,78.433367,71.091896
Shelton High School,Charter,"1,761","$1,056,600.00",$600.00,83.359455,83.725724,89.892107,92.617831,91.254969
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,64.746494,78.187702,71.467098
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,89.713896,93.392371,91.553134
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274201,83.989488,90.932983,93.254490,92.093736
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,89.558665,93.864370,91.711518
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.048432,81.033963,64.630225,79.300643,71.965434
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,90.632319,92.740047,91.686183
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,91.683992,92.203742,91.943867


# Top Performing Schools (by Passing Rate)

In [8]:
Top_5_df = schools_summ_present.sort_values("% Overall Passing Rate", ascending = False)
Top_5_df = Top_5_df.iloc[0:5,:]
del Top_5_df.index.name
Top_5_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274201,83.989488,90.932983,93.254490,92.093736
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,91.683992,92.203742,91.943867
Wright High School,Charter,"1,800","$1,049,400.00",$583.00,83.682222,83.955000,90.277778,93.444444,91.861111
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,89.558665,93.864370,91.711518
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,90.632319,92.740047,91.686183


# Bottom Performing Schools (by Passing Rate)

In [9]:
Bottom_5_df = schools_summ_present.sort_values("% Overall Passing Rate", ascending = True)
Bottom_5_df = Bottom_5_df.iloc[0:5,:]
del Bottom_5_df.index.name
Bottom_5_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.842711,80.744686,64.066017,77.744436,70.905226
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,63.318478,78.813850,71.066164
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,63.852132,78.281874,71.067003
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,63.750424,78.433367,71.091896
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,64.746494,78.187702,71.467098


# Average Math Scores by Grade

In [11]:
#Find the average math score grouped by school and grade
grade_students = students_df.groupby(["school", "grade"])
math_score_gs = grade_students.mean()
math_score_gs.reset_index(level=0, inplace=True)

#Use a loop to read the average scores into separated lists by grade
schools = []
grade_ten = []
grade_eleven = []
grade_twelve = []
grade_nine = []
for x in range(0,len(math_score_gs["math_score"]),4):
    schools.append(math_score_gs.iloc[x,0])
    grade_ten.append(math_score_gs.iloc[x,3])
    grade_eleven.append(math_score_gs.iloc[x+1,3])
    grade_twelve.append(math_score_gs.iloc[x+2,3])
    grade_nine.append(math_score_gs.iloc[x+3,3])

In [12]:
#Bundle those lists together in a new data frame
Math_grades = pd.DataFrame({"School": schools, "9th Grade": grade_nine, "10th Grade": grade_ten,
                            "11th Grade": grade_eleven, "12th Grade":grade_twelve})
Math_grades = Math_grades.set_index("School")
Math_grades = Math_grades[["9th Grade", "10th Grade", "11th Grade", "12th Grade"]]
del Math_grades.index.name
Math_grades

,9th Grade,10th Grade,11th Grade,12th Grade
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


# Average Reading Scores by Grade

In [13]:
#Find the average reading score grouped by school and grade
grade_students = students_df.groupby(["school", "grade"])
read_score_gs = grade_students.mean()
read_score_gs.reset_index(level=0, inplace=True)

#Use a loop to read the average scores into separated lists by grade
schools = []
grade_ten = []
grade_eleven = []
grade_twelve = []
grade_nine = []
for x in range(0,len(read_score_gs["reading_score"]),4):
    schools.append(read_score_gs.iloc[x,0])
    grade_ten.append(read_score_gs.iloc[x,2])
    grade_eleven.append(read_score_gs.iloc[x+1,2])
    grade_twelve.append(read_score_gs.iloc[x+2,2])
    grade_nine.append(read_score_gs.iloc[x+3,2])
    
#Bundle those lists together in a new data frame
Read_grades = pd.DataFrame({"School": schools, "9th Grade": grade_nine, "10th Grade": grade_ten,
                            "11th Grade": grade_eleven, "12th Grade":grade_twelve})
Read_grades = Read_grades.set_index("School")
Read_grades = Read_grades[["9th Grade", "10th Grade", "11th Grade", "12th Grade"]]
del Read_grades.index.name
Read_grades

,9th Grade,10th Grade,11th Grade,12th Grade
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


# Scores by School Spending

In [15]:
# Use the data frame schools_summ, create bins, then group the data by these bins
ss = schools_summ
bins = [560, 585, 610, 635, 660]
bin_names = ["\$560-\$585", "\$585-\$610", "\$610-\$635", "\$635-\$660"]
Binning = pd.cut(ss["Per Student Budget"], bins, labels=bin_names)
ss["Spending Range per Student"] = Binning
spending_bins = ss.groupby("Spending Range per Student")
spend_bin = spending_bins.mean()
spend_bin = spend_bin[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]]
spend_bin

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Range per Student,,,,,
\$560-\$585,83.455399,83.933814,90.350436,93.325838,91.838137
\$585-\$610,83.599686,83.885211,90.788049,92.410786,91.599418
\$610-\$635,80.199966,82.425360,77.172061,86.346507,81.759284
\$635-\$660,77.866721,81.368774,67.957362,80.268067,74.112715


# Scores by School Size

In [18]:
# Use the data frame schools_summ, create bins, then group the data by these bins
bins = [0, 1000, 2000, 5000]
bin_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
Binning = pd.cut(ss["Total Students"], bins, labels=bin_names)
ss["School Size"] = Binning
sizing_bins = ss.groupby("School Size")
size_bin = sizing_bins.mean()
size_bin = size_bin[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]]
size_bin

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,91.158155,92.471895,91.815025
Medium (1000-2000),83.374684,83.864438,89.931303,93.244843,91.588073
Large (2000-5000),77.746417,81.344493,67.631335,80.190800,73.911067


# Scores by School Type

In [19]:
# School type is already there, no need to bin, just groupby
type_bins = ss.groupby("School Type")
type_df = type_bins.mean()
type_df = type_df[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]]
type_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,90.363226,93.052812,91.708019
District,76.956733,80.966636,64.302528,78.324559,71.313543
